In [1]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Mounted at /gdrive
/gdrive


In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#from nltk.stemporter import PorterStemmer

# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")
!pip install mlxtend
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anuragbhattacharjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Read files
textfile = r'Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [3]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
print(textData.head())
print(y_train)

(2070, 16)
(2070, 2)
     ID                                           Comments
0  1309  Does not like the way the phone works. It is t...
1  3556  Wanted to know the nearest store location. Wan...
2  2230  Wants to know how to do text messaging. Referr...
3  2312  Asked how to disable call waiting. referred hi...
4  3327  Needs help learning how to use the phone. I su...
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


In [4]:
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv(r'TextDataTokenized1.csv')




In [5]:
# Use Snowball stemmer.
stemmer = SnowballStemmer("english")

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

export_csv = newTextData.to_csv(r'SnownewTextDataTS.csv')


In [6]:
# Use Porter stemmer.
stemmer_1 = PorterStemmer()

#Now do stemming - create a new dataframe to store stemmed version
newTextData_1=pd.DataFrame()
newTextData_1=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData_1['CommentsTokenizedStemmed_1'] = textData['CommentsTokenized'].apply(lambda x: [stemmer_1.stem(y) for y in x]) # Stem every word.

export_csv = newTextData.to_csv(r'Porter_newTextDataTS.csv')


In [7]:

#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

export_csv = newTextData.to_csv(r'newTextData-Joined.csv')

In [8]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
print(TD_counts.shape)
print(TD_counts.dtype)
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'TD_counts-TokenizedStemmed.csv')


(2070, 354)
int64
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effe

In [9]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF)
export_csv= DF_TF_IDF.to_csv(r'TFIDF_counts-TokenizedStemmed.csv')


(2070, 354)
      0    1    2    3        4    5    6    7    8         9    ...  344  \
0     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
1     0.0  0.0  0.0  0.0  0.27568  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
3     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
4     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
...   ...  ...  ...  ...      ...  ...  ...  ...  ...       ...  ...  ...   
2065  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2066  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2067  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2068  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.772949  ...  0.0   
2069  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   

      345  346       347  348  349  350  351  352  353  
0     

In [10]:
#Filter Method - Feature selection
#Suppose, we select 50 features with top 50 Fisher scores
selector = SelectKBest(k=50)
#selector = SelectKBest(score_func=chi2, k=50)

#new_DF_TF_IDF = SelectKBest(score_func=chi2, k=50).fit_transform(DF_TF_IDF,y_train)
new_DF_TF_IDF = selector.fit_transform(DF_TF_IDF,y_train)
print(new_DF_TF_IDF.shape)

feature_names_out = selector.get_support(indices=True)
print(feature_names_out)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)

export_csv= DF_TF_IDF_SelectedFeatures.to_csv(r'TFIDF_counts-Selected Features.csv')



(2070, 50)
[  0  14  35  49  50  51  62  64  70  72  81  99 109 115 118 121 130 145
 157 158 172 186 190 205 212 213 217 221 222 234 235 239 244 248 249 254
 259 264 266 273 307 312 313 315 319 325 328 338 342 350]
       0    1    2         3         4    5    6    7    8    9   ...   40  \
0     0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1     0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2     0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3     0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4     0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
...   ...  ...  ...       ...       ...  ...  ...  ...  ...  ...  ...  ...   
2065  0.0  0.0  0.0  0.000000  0.446161  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2066  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2067  0.0  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2068 

In [11]:
#Combine text data with Customer Data
print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
combined=pd.concat([X_train, DF_TF_IDF_SelectedFeatures], axis=1)
print(combined.shape)
print(combined)
export_csv= combined.to_csv(r'Combined-Cust+TFIDF+SelectedFeatures.csv')


(2070, 17)
(2070, 16)
(2070, 66)
        ID Sex Status  Children  Est_Income Car_Owner   Usage        Age  \
0        1   F      S         1    38000.00         N  229.64  24.393333   
1        6   M      M         2    29616.00         N   75.29  49.426667   
2        8   M      M         0    19732.80         N   47.25  50.673333   
3       11   M      S         2       96.33         N   59.01  56.473333   
4       14   F      M         2    52004.80         N   28.14  25.140000   
...    ...  ..    ...       ...         ...       ...     ...        ...   
2065  3821   F      S         0    78851.30         N   29.04  48.373333   
2066  3822   F      S         1    17540.70         Y   36.20  62.786667   
2067  3823   F      M         0    83891.90         Y   74.40  61.020000   
2068  3824   F      M         2    28220.80         N   38.95  38.766667   
2069  3825   F      S         0    28589.10         N  100.28  15.600000   

      RatePlan  LongDistance  ...   40   41   42   43 

In [12]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot = pd.get_dummies(combined,columns=X_cat)
print(combined_one_hot.shape)
export_csv= combined_one_hot.to_csv(r'combined_one_hot.csv')

['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 74)


In [13]:
# Split the combined data
X_train1, X_test1, y_train1, y_test1 = train_test_split(combined_one_hot, y_train, test_size = .20, 
                                                    stratify=y_train,random_state = 1)

In [14]:
#Construct a Random Forest Classifier on combined data and test accuracy
clf=RandomForestClassifier()
RF_Comb = clf.fit(X_train1,y_train1)
rf_predictions = clf.predict(X_test1)
print("Test Accuracy:", metrics.accuracy_score(y_test1,rf_predictions))
#print("Accuracy score (training): {0:.6f}".format(clf.score(X_test1, y_test1)))
print("Confusion Matrix:")
print(confusion_matrix(y_test1, rf_predictions))
print("Classification Report")
print(classification_report(y_test1, rf_predictions))


Test Accuracy: 0.8792270531400966
Confusion Matrix:
[[131  30]
 [ 20 233]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.87      0.81      0.84       161
     Current       0.89      0.92      0.90       253

    accuracy                           0.88       414
   macro avg       0.88      0.87      0.87       414
weighted avg       0.88      0.88      0.88       414



In [15]:
#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb, combined_one_hot, y_train, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')

=== All Accuracy Scores ===
[0.83584979 0.8554007  0.84630275 0.91153697 0.8625     0.909375
 0.821875   0.8234375  0.8703125  0.8890625  0.90119048 0.89325397
 0.94206349 0.8047619  0.90575397 0.83075397 0.89325397 0.95912698
 0.87738095 0.91488095]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8774036670054199




In [16]:
#Wrapper method -Do feature selection using a classification model
#clf = ExtraTreesClassifier(n_estimators=50)
clf = GradientBoostingClassifier(n_estimators=50)
#clf = DecisionTreeClassifier()
clf = clf.fit(DF_TF_IDF,y_train)
print(clf.feature_importances_)
#model = SelectFromModel(clf, prefit=True)
model = SelectFromModel(clf, prefit=True, max_features=7, threshold=-np.inf)
#model = SelectFromModel(clf, prefit=True)
X_new = model.transform(DF_TF_IDF)
X_new_SelectedFeatures= pd.DataFrame(X_new)
export_csv= X_new_SelectedFeatures.to_csv(r'X_new_SelectedFeatures.csv')

print(model.get_support())
print(X_new_SelectedFeatures)
#print(X_new_SelectedFeatures.shape)
#print(X_new_SelectedFeatures.head())


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.67729452e-03 0.00000000e+00 7.92863179e-03
 0.00000000e+00 0.00000000e+00 8.24927549e-02 0.00000000e+00
 3.72720690e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.16302064e-04 7.18165438e-03 9.95659538e-04
 0.00000000e+00 0.00000000e+00 1.50452100e-02 5.72930456e-06
 0.00000000e+00 0.00000000e+00 4.16022321e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.54193740e-03
 0.00000000e+00 2.17324479e-02 1.17299506e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.50019222e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.59113356e-04 0.00000000e+00
 0.00000000e+00 0.000000

FileNotFoundError: [Errno 2] No such file or directory: '/gdrive/My Drive/CIS508/Assignment_5/X_new_SelectedFeatures.csv'

In [17]:
#Combine new seelcted feature with Customer Data
print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
##combined=X_train.merge(DF_TF_IDF_SelectedFeatures, left_on='ID', right_on='ID')
#combined=pd.merge(X_train, DF_TF_IDF_SelectedFeatures, how='left', on=['ID', 'ID'])
#combined = pd.merge(X_train, DF_TF_IDF_SelectedFeatures, how='left',on = 'ID')
#combined=pd.merge(X_train, DF_TF_IDF_SelectedFeatures, join ='inner', on='ID')

combined_1=pd.concat([X_train, X_new_SelectedFeatures], axis=1)
print(combined_1.shape)
print(combined_1)
#export_csv= combined.to_csv(r'/gdrive/My Drive/CIS508/Assignment_5/Combined-Cust+TFIDF+SelectedFeatures.csv')

(2070, 17)
(2070, 16)
(2070, 23)
        ID Sex Status  Children  Est_Income Car_Owner   Usage        Age  \
0        1   F      S         1    38000.00         N  229.64  24.393333   
1        6   M      M         2    29616.00         N   75.29  49.426667   
2        8   M      M         0    19732.80         N   47.25  50.673333   
3       11   M      S         2       96.33         N   59.01  56.473333   
4       14   F      M         2    52004.80         N   28.14  25.140000   
...    ...  ..    ...       ...         ...       ...     ...        ...   
2065  3821   F      S         0    78851.30         N   29.04  48.373333   
2066  3822   F      S         1    17540.70         Y   36.20  62.786667   
2067  3823   F      M         0    83891.90         Y   74.40  61.020000   
2068  3824   F      M         2    28220.80         N   38.95  38.766667   
2069  3825   F      S         0    28589.10         N  100.28  15.600000   

      RatePlan  LongDistance  ...  Paymethod  LocalBil

In [18]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot_1 = pd.get_dummies(combined_1,columns=X_cat)
print(combined_one_hot.shape)

['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 74)


In [19]:
# Split the combined data
X_train1, X_test1, y_train1, y_test1 = train_test_split(combined_one_hot_1, y_train, test_size = .20, 
                                                    stratify=y_train,random_state = 1)

In [20]:
#Construct a Random Forest Classifier 
clf=RandomForestClassifier()
RF_Comb_1 = clf.fit(X_train1,y_train1)
rf_predictions = clf.predict(X_test1)
print("Test Accuracy:", metrics.accuracy_score(y_test1,rf_predictions))
#print("Accuracy score (training): {0:.6f}".format(clf.score(X_test1, y_test1)))
print("Confusion Matrix:")
print(confusion_matrix(y_test1, rf_predictions))
print("Classification Report")
print(classification_report(y_test1, rf_predictions))



Test Accuracy: 0.893719806763285
Confusion Matrix:
[[137  24]
 [ 20 233]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.87      0.85      0.86       161
     Current       0.91      0.92      0.91       253

    accuracy                           0.89       414
   macro avg       0.89      0.89      0.89       414
weighted avg       0.89      0.89      0.89       414



In [21]:
#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb_1, combined_one_hot_1, y_train, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')

=== All Accuracy Scores ===
[0.85598142 0.8554007  0.84630275 0.89566396 0.8828125  0.9171875
 0.83125    0.8109375  0.8671875  0.86875    0.88869048 0.89781746
 0.91369048 0.7968254  0.90575397 0.83075397 0.89781746 0.95912698
 0.90238095 0.90238095]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8763355957220286




In [22]:
#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y_train, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')

=== All Accuracy Scores ===
[0.50483933 0.49690283 0.51645374 0.53290747 0.496875   0.546875
 0.5390625  0.5234375  0.509375   0.546875   0.54206349 0.52619048
 0.49662698 0.525      0.50119048 0.50456349 0.5125     0.53869048
 0.47738095 0.50119048]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.5169500096786682




In [23]:
#Construct a Random Forest Classifier WITHOUT text data
print(CustInfoData.shape)
X_train1=combined_one_hot.iloc[:,1:10]
X_train2=combined_one_hot.iloc[:,60:]
print(X_train1.shape)
print(X_train1.head())
print(X_train2.shape)
print(X_train2.head())
combined1=pd.concat([X_train1, X_train2], axis=1)
print(combined1.shape)
print(combined1.head())


(2070, 17)
(2070, 9)
   Children  Est_Income   Usage        Age  RatePlan  LongDistance  \
0         1    38000.00  229.64  24.393333         3         23.56   
1         2    29616.00   75.29  49.426667         2         29.78   
2         0    19732.80   47.25  50.673333         3         24.81   
3         2       96.33   59.01  56.473333         1         26.13   
4         2    52004.80   28.14  25.140000         1          5.03   

   International   Local  Dropped  
0            0.0  206.08        0  
1            0.0   45.50        0  
2            0.0   22.44        0  
3            0.0   32.88        1  
4            0.0   23.11        0  
(2070, 14)
   Sex_F  Sex_M  Status_D  Status_M  Status_S  Car_Owner_N  Car_Owner_Y  \
0      1      0         0         0         1            1            0   
1      0      1         0         1         0            1            0   
2      0      1         0         1         0            1            0   
3      0      1         0      

In [24]:
#Construct a Random Forest Classifier WITHOUT text data

rf_NT=clf.fit(combined1,y_train)
print("Accuracy score (training): {0:.6f}".format(rf_NT.score(combined1, y_train)))
rf_predictions = rf_NT.predict(combined1)
print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predictions))
print("Classification Report")
print(classification_report(y_train, rf_predictions))


Accuracy score (training): 0.957488
Confusion Matrix:
[[ 760   44]
 [  44 1222]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.95      0.95      0.95       804
     Current       0.97      0.97      0.97      1266

    accuracy                           0.96      2070
   macro avg       0.96      0.96      0.96      2070
weighted avg       0.96      0.96      0.96      2070



In [25]:
#run cross-validation - WITHOUT Text Data
rf_NT_cv_score = cross_val_score(rf_NT, combined1, y_train, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_NT_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - WITHOUT Text: ",rf_NT_cv_score.mean())
print('\n')


=== All Accuracy Scores ===
[0.84436702 0.89198606 0.89508324 0.91153697 0.890625   0.934375
 0.8515625  0.81875    0.8671875  0.90625    0.91369048 0.90119048
 0.93075397 0.8172619  0.90575397 0.79781746 0.88988095 0.93869048
 0.88194444 0.93075397]


=== Mean Accuracy Score ===
Mean Accuracy Score - WITHOUT Text:  0.8859730691056911


